<a href="https://colab.research.google.com/github/faezesarlakifar/SBU-NLPLab-Internship/blob/main/DataCrawling_ACL_Anthology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [17]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
import os 
from tqdm import tqdm
import csv
import lxml 

## Define Constants 

In [6]:
page_url = "https://aclanthology.org/events/acl-2022"
conf_name = 'acl_2022_articles'
conf_id = '2022-acl-long'

In [7]:
page_url = "https://aclanthology.org/events/acl-2022"
conf_name = 'acl_2022_articles-1'
conf_id = '2022-acl-short' 

In [17]:
page_url = "https://aclanthology.org/events/acl-2022"
conf_name = 'acl_2022_articles-2'
conf_id = '2022-acl-srw' 

In [22]:
conf_name = 'acl_2022_articles-3'
conf_id = '2022-acl-demo' 

In [29]:
conf_name = 'acl_2022_articles-4'
conf_id = '2022-acl-tutorials'

In [39]:
conf_name = 'acl_2022_articles-5'
conf_id = '2022-bigscience-1'

In [7]:
conf_name = 'acl_2022_articles-6'
conf_id = '2022-bionlp-1' 

In [ ]:
conf_name = 'acl_2022_articles-7'
conf_id = '2022-cmcl-1' 

In [ ]:
conf_name = 'acl_2022_articles-8'
conf_id = '2022-computel-1'

In [ ]:
conf_name = 'acl_2022_articles-9'
conf_id = '2022-constraint-1'

In [18]:
page_url = "https://aclanthology.org/events/acl-2022"
conf_name = 'acl_2022_articles-final'

conf_ids = ['2022-bionlp-1', '2022-cmcl-1', '2022-computel-1', '2022-constraint-1', '2022-csrr-1',
'2022-deelio-1', '2022-dialdoc-1', '2022-dravidianlangtech-1', '2022-ecnlp-1', '2022-fever-1',
'2022-findings-acl', '2022-fl4nlp-1', '2022-humeval-1', '2022-in2writing-1', '2022-spnlp-1', '2022-wassa-1', '2022-wit-1']

In [27]:
html_doc = requests.get(page_url).text
soup = BeautifulSoup(html_doc, 'html.parser')

paper_lists = []

for item in conf_ids:

    conf_id = item

    print( conf_id)

    main_papers = soup.find('div', id = conf_id).find_all('p', class_ = "d-sm-flex")

    paper_list = []

    counter = 0
    for paper_p in main_papers:
        pdf_url = paper_p.contents[0].contents[0]['href']
        paper_span = paper_p.contents[-1]
        assert paper_span.name == 'span'
        paper_a = paper_span.strong.a
        title = paper_a.get_text()
        strs = paper_span.get_text().split('|')
        authors = strs[1:]
        
        
        cleaned_authors = []
        for author in authors:
            author = author.strip('\n')
            cleaned_authors.append(author)
            
        authors = ', '.join(cleaned_authors) 
        
        query = "allintitle: "
        query += title
        
        if(len(cleaned_authors) != 0):
            query += " author: "
            author_ = cleaned_authors[0].split(' ')
            query += author_[0]

        url = "https://aclanthology.org" + paper_a['href']
        
        headers = {
        'User-agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"   
        }
        
        params = {
            "q": query 
        }

        html = requests.get('https://scholar.google.com/scholar', headers=headers, params=params).text
        soup_1 = BeautifulSoup(html, 'lxml')

        citation = "0"
        for result in soup_1.select('.gs_ri'):
            citation_finder = result.select('.gs_fl')
            strs = []
            
            for x in citation_finder:
                strs.append(str(x))

            citation_finder = ''.join(strs)
            #print(citation_finder)
    
            try:
                finded_index = citation_finder.find('Cited by')
                #print(finded_index) 
                finded_index  += 9 
                #print(citation_finder[finded_index])
                if(citation_finder[finded_index].isdigit()):
                    citation = str(citation_finder[finded_index]) 
                    finded = finded_index + 1
                    #print(finded)
                    for i in range(len(citation_finder)-finded_index ):
                        if(citation_finder[finded].isdigit() == False):
                            break
                        else:
                            citation += citation_finder[finded]
                            finded += 1
                else:
                    citation = "0"
            except: 
                citation = "0"


        paper_list.append([title, url, pdf_url, authors, citation])
    
    print('There is {} papers now'.format(len(paper_list))) 
    
    paper_lists.append(paper_list)
    

2022-bionlp-1
There is 44 papers now
2022-cmcl-1
There is 17 papers now
2022-computel-1
There is 24 papers now
2022-constraint-1
There is 12 papers now
2022-csrr-1
There is 8 papers now
2022-deelio-1
There is 11 papers now
2022-dialdoc-1
There is 22 papers now
2022-dravidianlangtech-1
There is 45 papers now
2022-ecnlp-1
There is 30 papers now
2022-fever-1
There is 9 papers now
2022-findings-acl
There is 332 papers now
2022-fl4nlp-1
There is 5 papers now
2022-humeval-1
There is 11 papers now
2022-in2writing-1
There is 15 papers now
2022-spnlp-1
There is 8 papers now
2022-wassa-1
There is 36 papers now
2022-wit-1
There is 4 papers now


In [30]:
final_paper_list = []
for list_ in paper_lists:
    final_paper_list.extend(list_)

In [31]:
print('There are total {} papers'.format(len(final_paper_list))) 

There are total 633 papers


In [32]:
paper_lists = final_paper_list

## Make a .csv file 

In [34]:
fields = ['Title', 'URL', 'PDF_URL', 'Authors', 'Citation'] 

name = conf_name + ".csv" 

with open(name, 'w', encoding='utf8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(paper_lists)

## Download and save full PDFs

In [35]:
if not os.path.exists(conf_name):
    os.mkdir(conf_name)

illegal_chr = r'\/:*?<>|'
table = ''.maketrans('', '', illegal_chr)
for i, paper in tqdm(list(enumerate(paper_lists))):
    r = requests.get(paper[2])
    n = '{}.{}.pdf'.format(i+1, paper[0].translate(table))
    with open('./{}/{}'.format(conf_name, n), 'wb') as f:
        f.write(r.content) 

  3%|▎         | 19/633 [00:33<17:00,  1.66s/it]